In [4]:


from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# Colab-safe ROOT + folders
# -----------------------------
BPS_DENOM = 10_000

ROOT = Path(os.getcwd())          # ✅ In Colab this is typically /content
DATA_DIR = ROOT / "data"
OUT_DIR = ROOT / "outputs"
DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)


# -----------------------------
# Model (drivers + P&L)
# -----------------------------
@dataclass
class FintechDrivers:
    tpv_usd: float
    txn_count: float
    take_rate_bps: float
    revenue_per_txn: float
    network_cost_bps: float
    processing_cost_per_txn: float
    loss_rate_bps: float
    opex_fixed_usd: float
    opex_variable_pct_rev: float


def compute_pnl(d: FintechDrivers) -> dict:
    """
    Fintech unit economics P&L:

    Revenue = TPV * take_rate_bps/10,000 + txn_count * revenue_per_txn
    Direct costs = TPV * network_cost_bps/10,000 + txn_count * processing_cost_per_txn
    Losses = TPV * loss_rate_bps/10,000

    Gross Profit = Revenue - Direct costs - Losses
    Contribution Profit = Gross Profit - Variable Opex
    EBIT = Contribution Profit - Fixed Opex
    """
    revenue = d.tpv_usd * (d.take_rate_bps / BPS_DENOM) + d.txn_count * d.revenue_per_txn
    direct_costs = d.tpv_usd * (d.network_cost_bps / BPS_DENOM) + d.txn_count * d.processing_cost_per_txn
    losses = d.tpv_usd * (d.loss_rate_bps / BPS_DENOM)

    gross_profit = revenue - direct_costs - losses
    gross_margin = gross_profit / revenue if revenue else 0.0

    variable_opex = revenue * d.opex_variable_pct_rev
    contribution_profit = gross_profit - variable_opex
    contribution_margin = contribution_profit / revenue if revenue else 0.0

    ebit = contribution_profit - d.opex_fixed_usd
    operating_margin = ebit / revenue if revenue else 0.0

    # Unit metrics for fintech storytelling
    rev_bps = (revenue / d.tpv_usd) * BPS_DENOM if d.tpv_usd else 0.0
    cost_plus_loss_bps = ((direct_costs + losses) / d.tpv_usd) * BPS_DENOM if d.tpv_usd else 0.0

    return {
        "Revenue": revenue,
        "Direct_Costs": direct_costs,
        "Losses": losses,
        "Gross_Profit": gross_profit,
        "Gross_Margin": gross_margin,
        "Variable_Opex": variable_opex,
        "Contribution_Profit": contribution_profit,
        "Contribution_Margin": contribution_margin,
        "Fixed_Opex": d.opex_fixed_usd,
        "EBIT": ebit,
        "Operating_Margin": operating_margin,
        "Rev_bps_of_TPV": rev_bps,
        "CostPlusLoss_bps_of_TPV": cost_plus_loss_bps,
    }


def build_pnl_table(scenarios_df: pd.DataFrame) -> pd.DataFrame:
    """
    Each row in scenarios_df is a scenario. Return P&L lines + margins per scenario.
    """
    out_rows = []
    for _, r in scenarios_df.iterrows():
        d = FintechDrivers(
            tpv_usd=float(r["tpv_usd"]),
            txn_count=float(r["txn_count"]),
            take_rate_bps=float(r["take_rate_bps"]),
            revenue_per_txn=float(r["revenue_per_txn"]),
            network_cost_bps=float(r["network_cost_bps"]),
            processing_cost_per_txn=float(r["processing_cost_per_txn"]),
            loss_rate_bps=float(r["loss_rate_bps"]),
            opex_fixed_usd=float(r["opex_fixed_usd"]),
            opex_variable_pct_rev=float(r["opex_variable_pct_rev"]),
        )
        pnl = compute_pnl(d)
        pnl["scenario"] = r["scenario"]
        out_rows.append(pnl)

    pnl_df = pd.DataFrame(out_rows)
    # Put scenario first
    cols = ["scenario"] + [c for c in pnl_df.columns if c != "scenario"]
    return pnl_df[cols]


# -----------------------------
# Data generation (template + sample scenarios)
# -----------------------------
def write_input_files() -> None:
    """
    Writes:
      data/input_template.csv
      data/sample_company_data.csv
    """
    input_template = pd.DataFrame(
        [
            ["base", "tpv_usd", 1_000_000_000, "Annual total payment volume processed ($)"],
            ["base", "txn_count", 25_000_000, "Annual transaction count"],
            ["base", "take_rate_bps", 85, "Blended take rate (bps of TPV)"],
            ["base", "revenue_per_txn", 0.03, "Additional revenue per txn ($)"],
            ["base", "network_cost_bps", 42, "Network/interchange/partner cost (bps of TPV)"],
            ["base", "processing_cost_per_txn", 0.07, "Processing cost per txn ($)"],
            ["base", "loss_rate_bps", 18, "Losses: fraud/chargebacks/credit losses (bps of TPV)"],
            ["base", "opex_fixed_usd", 55_000_000, "Fixed opex (annual $)"],
            ["base", "opex_variable_pct_rev", 0.18, "Variable opex as % of revenue (0-1)"],
        ],
        columns=["scenario", "driver", "value", "notes"],
    )
    input_template.to_csv(DATA_DIR / "input_template.csv", index=False)

    sample_company_data = pd.DataFrame(
        [
            # scenario, tpv, txns, take_rate, rev/txn, network_bps, proc/txn, loss_bps, fixed_opex, var_opex_pct
            ["Base",     1_000_000_000, 25_000_000, 85, 0.030, 42, 0.070, 18, 55_000_000, 0.18],
            ["Best",     1_120_000_000, 27_500_000, 90, 0.035, 40, 0.065, 14, 54_000_000, 0.17],
            ["Downside",   920_000_000, 23_000_000, 82, 0.028, 45, 0.075, 26, 57_000_000, 0.19],
            ["Fee_Shock", 1_000_000_000, 25_000_000, 85, 0.030, 48, 0.070, 18, 55_000_000, 0.18],
            ["Loss_Shock",1_000_000_000, 25_000_000, 85, 0.030, 42, 0.070, 30, 55_000_000, 0.18],
        ],
        columns=[
            "scenario",
            "tpv_usd",
            "txn_count",
            "take_rate_bps",
            "revenue_per_txn",
            "network_cost_bps",
            "processing_cost_per_txn",
            "loss_rate_bps",
            "opex_fixed_usd",
            "opex_variable_pct_rev",
        ],
    )
    sample_company_data.to_csv(DATA_DIR / "sample_company_data.csv", index=False)


# -----------------------------
# Sensitivity (two-way heatmap)
# -----------------------------
def two_way_heatmap_take_vs_loss(base: FintechDrivers, take_rates_bps, loss_rates_bps) -> pd.DataFrame:
    """
    Matrix of Operating Margin for take rate (x) vs loss rate (y).
    """
    grid = []
    for loss_bps in loss_rates_bps:
        row = []
        for take_bps in take_rates_bps:
            d = FintechDrivers(**base.__dict__)
            d.take_rate_bps = float(take_bps)
            d.loss_rate_bps = float(loss_bps)
            pnl = compute_pnl(d)
            row.append(pnl["Operating_Margin"])
        grid.append(row)

    return pd.DataFrame(grid, index=loss_rates_bps, columns=take_rates_bps)


def save_heatmap(df: pd.DataFrame, title: str, xlabel: str, ylabel: str, outpath: Path) -> None:
    fig, ax = plt.subplots(figsize=(9, 5))
    im = ax.imshow(df.values, aspect="auto")

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    ax.set_xticks(range(len(df.columns)))
    ax.set_xticklabels([str(c) for c in df.columns], rotation=45, ha="right")

    ax.set_yticks(range(len(df.index)))
    ax.set_yticklabels([str(i) for i in df.index])

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout()
    fig.savefig(outpath, dpi=200)
    plt.close(fig)


# -----------------------------
# Executive summary output
# -----------------------------
def write_exec_summary(pnl: pd.DataFrame) -> None:
    """
    Writes outputs/executive_summary.md using Base/Best/Downside.
    """
    def get_row(name: str):
        sub = pnl[pnl["scenario"] == name]
        if sub.empty:
            return None
        return sub.iloc[0]

    base = get_row("Base")
    best = get_row("Best")
    down = get_row("Downside")

    lines = ["# Executive Summary (Auto)\n"]
    if base is not None:
        lines.append(f"- Base Revenue: ${base['Revenue']:,.0f}; Operating Margin: {base['Operating_Margin']*100:,.1f}%")
        lines.append(
            f"- Unit economics (Base): Revenue bps of TPV = {base['Rev_bps_of_TPV']:,.1f} bps; "
            f"Cost+Loss bps of TPV = {base['CostPlusLoss_bps_of_TPV']:,.1f} bps."
        )

    if best is not None and base is not None:
        lines.append(
            f"- Best-case OM: {best['Operating_Margin']*100:,.1f}% "
            f"(Δ {((best['Operating_Margin'] - base['Operating_Margin'])*100):+.1f} pts vs Base) "
            f"driven by higher TPV/take rate and lower network/loss costs."
        )

    if down is not None and base is not None:
        lines.append(
            f"- Downside OM: {down['Operating_Margin']*100:,.1f}% "
            f"(Δ {((down['Operating_Margin'] - base['Operating_Margin'])*100):+.1f} pts vs Base) "
            f"driven by higher loss rate/network fees and weaker TPV."
        )

    lines.append("- Next: extend sensitivities to (TPV vs network_cost_bps) and add a target-margin backsolve for pricing/volume.")

    (OUT_DIR / "executive_summary.md").write_text("\n".join(lines), encoding="utf-8")


# -----------------------------
# Main runner
# -----------------------------
def main() -> None:
    # 1) Generate input CSVs
    write_input_files()

    # 2) Load scenarios + compute P&L table
    scenarios = pd.read_csv(DATA_DIR / "sample_company_data.csv")
    pnl = build_pnl_table(scenarios)

    # 3) Save scenario summary
    pnl.to_csv(OUT_DIR / "scenario_summary.csv", index=False)

    # 4) Save executive summary
    write_exec_summary(pnl)

    # 5) Two-way sensitivity heatmap (Take Rate vs Loss Rate)
    base_row = scenarios[scenarios["scenario"] == "Base"].iloc[0]
    base = FintechDrivers(
        tpv_usd=float(base_row["tpv_usd"]),
        txn_count=float(base_row["txn_count"]),
        take_rate_bps=float(base_row["take_rate_bps"]),
        revenue_per_txn=float(base_row["revenue_per_txn"]),
        network_cost_bps=float(base_row["network_cost_bps"]),
        processing_cost_per_txn=float(base_row["processing_cost_per_txn"]),
        loss_rate_bps=float(base_row["loss_rate_bps"]),
        opex_fixed_usd=float(base_row["opex_fixed_usd"]),
        opex_variable_pct_rev=float(base_row["opex_variable_pct_rev"]),
    )

    take_rates = [75, 80, 85, 90, 95]
    loss_rates = [12, 16, 20, 24, 28]

    hm = two_way_heatmap_take_vs_loss(base, take_rates, loss_rates)
    heatmap_path = OUT_DIR / "sensitivity_heatmap_take_vs_loss.png"
    save_heatmap(
        hm,
        title="Operating Margin: Take Rate vs Loss Rate",
        xlabel="Take Rate (bps)",
        ylabel="Loss Rate (bps)",
        outpath=heatmap_path,
    )

    # 6) Print locations (useful in Colab)
    print("✅ Done. Files created here:")
    print("ROOT:", ROOT)
    print("DATA_DIR:", DATA_DIR)
    print("OUT_DIR:", OUT_DIR)
    print(" -", DATA_DIR / "input_template.csv")
    print(" -", DATA_DIR / "sample_company_data.csv")
    print(" -", OUT_DIR / "scenario_summary.csv")
    print(" -", OUT_DIR / "executive_summary.md")
    print(" -", heatmap_path)


# Execute
if __name__ == "__main__":
    main()


✅ Done. Files created here:
ROOT: /content
DATA_DIR: /content/data
OUT_DIR: /content/outputs
 - /content/data/input_template.csv
 - /content/data/sample_company_data.csv
 - /content/outputs/scenario_summary.csv
 - /content/outputs/executive_summary.md
 - /content/outputs/sensitivity_heatmap_take_vs_loss.png
